In [1]:
# Imports required libaries
import pandas as pd
import numpy as np
import re

import spacy
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
# Load spacy 
nlp = spacy.load("en_core_web_sm")

In [3]:
sc_india_1999 = pd.read_csv('Supreme_Court_cases_India_tokens_above_1999_final.csv')
sc_india_1999.drop(sc_india_1999.filter(regex="Unname"),axis=1, inplace=True)
sc_india_1999.head()

,Document Id,Tokens,Corpus
0,484625,"['littttttj', 'maintainedbeforethe', 'motor', ...",littttttj maintainedbeforethe motor accident t...
1,60864,"['littttttj', 'sethijthe', 'complaint', 'thene...",littttttj sethijthe complaint thenegotiable in...
2,1373794,"['complaint', 'forwardedby', 'magistrate', 'po...",complaint forwardedby magistrate police regist...
3,55804,"['littttttj', 'sye', 'quadri', 'jthis', 'benga...",littttttj sye quadri jthis bengal electricity ...
4,301256,"['litttttttjsethijdelay', 'condonedleave', 'gr...",litttttttjsethijdelay condonedleave grantedthe...


In [4]:
sc_india_1999.shape

(20269, 3)

In [5]:
sc_india_1999.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20269 entries, 0 to 20268
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Document Id  20269 non-null  int64 
 1   Tokens       20269 non-null  object
 2   Corpus       20234 non-null  object
dtypes: int64(1), object(2)
memory usage: 475.2+ KB


In [6]:
sc_india_1999.dropna(inplace=True)

In [7]:
# tfidf_vectorizer
max_df = 9.09
min_df = 0.07
features = 3000

tfidf_vectorizer = TfidfVectorizer(max_df = max_df, min_df = min_df,  max_features = features)
tfidf_matrix = tfidf_vectorizer.fit_transform(sc_india_1999['Corpus'])

sc_india_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names())
sc_india_matrix

,able,absence,absolute,absolutely,abuse,accept,acceptance,accompany,accord,accordance,...,wide,withdraw,withthe,witness,word,work,writ,write,writing,wrong
0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.014558,0.017726,...,0.000000,0.000000,0.053341,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
1,0.0,0.014353,0.0,0.000000,0.0,0.011007,0.000000,0.0,0.022146,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.026636,0.000000,0.0,0.014647,0.022535,0.0
2,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.015058,0.015072,0.0,0.000000,0.000000,0.0
3,0.0,0.006481,0.0,0.000000,0.0,0.019879,0.030232,0.0,0.005000,0.024350,...,0.000000,0.008915,0.000000,0.000000,0.072160,0.174550,0.0,0.026454,0.040698,0.0
4,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,...,0.112105,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20229,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.022689,0.000000,...,0.000000,0.000000,0.000000,0.167605,0.013644,0.000000,0.0,0.015006,0.000000,0.0
20230,0.0,0.013731,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.010593,0.000000,...,0.000000,0.000000,0.000000,0.071136,0.000000,0.000000,0.0,0.014012,0.000000,0.0
20231,0.0,0.000000,0.0,0.034837,0.0,0.000000,0.000000,0.0,0.000000,0.011240,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
20232,0.0,0.000000,0.0,0.000000,0.0,0.013335,0.000000,0.0,0.000000,0.000000,...,0.000000,0.047842,0.000000,0.000000,0.048403,0.000000,0.0,0.000000,0.000000,0.0


In [10]:
lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(sc_india_matrix)
lda.components_

array([[13.63921478, 27.49120153, 23.97138786, ..., 69.57678176,
        11.52696366, 19.43006645],
       [ 0.10004098,  1.36254747,  0.10025175, ...,  0.92202783,
         0.10001008,  0.10028527],
       [ 0.10001069,  0.10000458,  0.10000748, ...,  0.10000825,
         0.10000611,  0.10000861],
       ...,
       [ 0.10000118,  0.10000135,  0.10000093, ...,  0.10000092,
         0.10000054,  0.10000095],
       [ 4.22258286,  8.4811536 ,  2.95180471, ...,  9.28501623,
         3.49779024,  4.50537858],
       [ 0.10000116,  0.10000126,  0.10000083, ...,  0.10000085,
         0.10000109,  0.10000075]])

In [11]:
def lda_topics(corpus):
    max_df = 9.09
    min_df = 0.07
    features = 3000
    top_words = 20
    
    tfidf_vectorizer = TfidfVectorizer(max_df = max_df, min_df = min_df, max_features = features)
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
    sc_india_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names())
    
    lda = LatentDirichletAllocation(max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(sc_india_matrix)
    tfidf_features = tfidf_vectorizer.get_feature_names()
    
    topicdict = {}
    for idx, topic in enumerate(lda.components_):
        topicdict[idx]=" ".join([tfidf_features[i] for i in topic.argsort()[:-top_words -1:-1]])
    return tfidf_matrix, lda, topicdict

In [12]:
tfidf_matrix, lda_test, ldaword = lda_topics(sc_india_1999['Corpus'])

In [14]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

#print topic words    
print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(lda_test, tfidf_feature_names,20)


Topics in NMF model:
Topic #0:
land property respondent appellant plaintiff order high decree date possession agreement party sale acquisition rent deed award application judgment learn
Topic #1:
order high petition learn appellant counsel pass hear respondent dispose date party writ civil application matter impugned direct accordingly disposed
Topic #2:
council bar advocate committee complainant guilty punishment complaint practice cease strike remove withthe substitute allegation large appellant association properly chapter
Topic #3:
compensation vehicle rs award tribunal commission loss national appellant company deceased forum income pay policy high owner respondent liability injury
Topic #4:
service respondent order high appellant date writ post petition employee appointment candidate issue authority committee tribunal hold article learn shall
Topic #5:
accuse offence criminal appellant sentence bail high order magistrate complaint trial code police complainant respondent learn i